Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your collaborators below:

In [ ]:
COLLABORATORS = ""

---

In [ ]:
import numpy as np

In Problem 4 we looked at how we can use hidden Markov models to model the structure of human language. In this challenge problem we will investigate a more complex class of probabilistic language models: probabibilistic context-free grammars. You will 1) use recursion to sample a sentence from the provided grammar and 2) use dynamic programming to find the highest probability parse tree for a given sentence from the grammar.

Recall that a context-free grammar (CFG) is a formalism that compactly descibes the set of acceptable sentences in a language. A CFG consists of a set of rewrite rules that specify how to transform nonterminals into other nonterminals or terminals; these re-write rules are applied recursively until only a string of nonterminals remains. Rules that might generate a toy language resembling a subset of English might contain, for example, the first columns in the following tables:

<table class="table table-striped" style="width: 18em;">
    <tr>
        <td colspan=2><center>Nonterminals</center>
        </td>
    </tr>    
    <tr>
        <td>Rule</td><td>Probability</td>
    </tr>        
    <tr>    
    <td>S -> NP VP</td><td>1</td>
    </tr>
    <tr>
        <td>NP -> DET N</td><td>.7</td>
    </tr>
    <tr>
        <td>NP -> PPN N</td><td>.3</td>
    </tr>
    <tr>
        <td>VP -> ADV V</td><td>.5</td>
    </tr>
    <tr>
        <td>VP -> V PP</td><td>.4</td>
    </tr>
    <tr>
        <td>VP -> V ADV</td><td>.1</td>
    </tr>
    <tr>
        <td>PP -> P NP</td><td>1</td>
    </tr>
</table>

<table class="table table-striped" style="width: 18em;">
    
    <tr>
        <td colspan=2><center>Terminals</center>
        </td>
    </tr>    
    <tr>
        <td>Rule</td><td>Probability</td>
    </tr>
    <tr>
        <td>N -> cat</td><td>.5</td>    
    </tr>
    <tr>
        <td>N -> dog</td><td>.5</td>    
    </tr>
    <tr>
        <td>V -> runs</td><td>.8</td>    
    </tr>
    <tr>
        <td>V -> barks</td><td>.2</td>    
    </tr>
    <tr>
        <td>ADJ -> black</td><td>.7</td>
    </tr>
    <tr>
        <td>ADJ -> white</td><td>.3</td>
    </tr>
    <tr>
        <td>ADV -> quickly</td><td>.1</td>
    </tr>
    <tr>
        <td>ADV -> slowly</td><td>.9</td>    
    </tr>
    <tr>
        <td>DET -> the</td><td>.6</td>
    </tr>
    <tr>
        <td>DET -> a</td><td>.4</td>
    </tr>
    <tr>
        <td>PPN -> his</td><td>.5</td>
    </tr>
    <tr>
        <td>PPN -> her</td><td>.5</td>
    </tr>
    <tr>
        <td>P -> by</td><td>.6</td>
    </tr>
    <tr>
        <td>P -> with</td><td>.4</td>
    </tr>
</table>

In these tables S represents a sentence node, NP a noun phrase, VP a verb phrase, PP a prepositional phrase, N a noun, V a verb, ADJ an Adjective, DET a determiner, PPN a personal pronoun, and P a preposition. This CFG specifies that "his cat runs with a dog" is in the language, but that "dog cat his quickly" is not.

A probabilistic context-free grammar (PCFG) is CFG which additionally provides the probability of each rule expansion (i.e. column 2 of the above tables). A PCFG can specify not only that the sentence can be generated, but also that there are different probabilities associated with each tree structure. Note that probabilities expansion rules for a symbol sum to 1, for example, P(DET-> the) + P(DET -> a) = 1.

Run the following cell to define the variable `pcfg`.

In [ ]:
pcfg = [ ('S',['NP','VP'], 1), 
    ('NP',['DET','N'], .7),
    ('NP',['PPN','N'], .3),    
    ('VP',['ADV','V'], .4),
    ('VP', ['V', 'PP'], .5),
    ('VP',['V', 'ADV'], .1),    
    ('PP',['P', 'NP'], 1),

    #terminals
    ('N',['cat'], .5),    
    ('N',['dog'], .5),    
    ('V',['runs'], .8),    
    ('V',['barks'], .2),    
    ('ADJ',['black'], .7),
    ('ADV',['quickly'], .1),
    ('ADV',['slowly'], .9),    
    ('ADJ',['white'], .3),    
    ('DET',['the'], .6),
    ('DET',['a'], .4),
    ('PPN',['his'], .5),
    ('PPN',['her'], .5),
    ('P',['by'], .6),
    ('P',['with'], .4),
]

Note that the rules here all have binary expansions—there are always two terms on the right hand side, like NP -> DET N, S -> NP VP. While this limits the expressive capacities of the grammar, it simplifies the computations in Part B.

---
## Part A

Using the probabilities in the PCFG, it is possible to sample from the probability distribution over sentences in the langauge. This is done by applying the rewrite rules according to probabilities listed in the PCFG.

<div class="alert alert-success">Complete the function `rewriteString` that outputs a sentence and a probability according to the probabilities of the provided PCFG. Remember that we can think of a PCFG as a set of rewrite rules for transforming a start symbol, in this case S, into a set of terminals. `rewriteString` should be recursively defined: the function should call itself in the process of rewriting the string. You will modify `sentence` and `probs` each time the function is called. You may wish to use `np.random.choice` to choose among a set of options weighed by a vector of probabilities.</div>

<div class="alert alert-danger">`rewriteString` should be a stochastic function that produces a variety of different output sentences based on chance. Intuitively, it should produce the most probable sentences more frequently than the least probable sentences.</div>

In [ ]:
def rewriteString(sentence, probs, pcfg):        
    """
    Constructs a sentence according to the probabilities in the PCFG
    
    Parameters
    ----------
    
    sentence: string
        String to rewrite (this will be the start symbol 
        'S' when first called)
    probs: list
        list of probabilities of expansions (this will be an 
        empty list when first called)
    pcfg: list of tuples that encodes the grammar
        Each list item is a tuple of (input, output, probability)
        input is a string, output is a list of two strings, and
        probability is a float
 
    Returns
    -------
    a tuple with a sentence with each word as an entry in a list
    and the probability as a float
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(rewriteString(['S'],[], pcfg))

In [ ]:
# add your own test cases here!

In [ ]:
from nose.tools import assert_equal

for i in range(50):
    testSentence = (rewriteString(['S'],[], pcfg))
    assert(type(testSentence[0]) is list)
    assert(isinstance(testSentence[1], float))
    assert(len(testSentence[0]) >= 4)
    assert(testSentence[1] < .1)

rewriteStringSuccess = True    
    
print("Success!")    

---
## Part B

While a CFG allows us to find all of the possible parses of a given sentence, a PCFG additionally allows us to assign probabilities to all of the parses. The CYK (or alternatively CKY) algorithm uses dynamic programming to search over the space of possible trees, either aggregating probabilities (the forward algorithm) or finding the best parse (Viterbi). Here you will  implement a parser that uses CYK to find the best parse of a given sentence.

Consult the following resources for information on the motivation and implementation of the CYK algorithm:

*Overview of Statistical Parsing (powerpoint)* <br />
http://www.cs.utexas.edu/~mooney/cs388/slides/stats-parsing.ppt<br />
*Explanation and Pseudocode for CYK (video)*<br />
https://www.youtube.com/watch?v=hq80J8kBg-Y<br />
*Worked example for CYK (video)* <br />
https://www.youtube.com/watch?v=MiEKnFyErbQ<br />

<div class="alert alert-danger">Note that the grammar provided above contains only binary expansion rules (e.g. S -> NP VP), which means you do not need to build in rules to handle unary rewrite rules (e.g. NP -> N). This should simplify the algorithm considerably.</div>

We have provided the function `makeTree`, as well as defined some of the data types you should use within `getHighestProbParse`. In `getHighestProbParse`, `score` and `back` are both n-by-n "charts" (see above references) where each entry is a dictionary; each dictionary contains all nonterminals as keys. For `score`, the values of the dictionaries are initialized to 0; some of these values need to be updated as you fill out the chart. `back` should contain a list of corresponding backtraces so that you can reconstruct the best parse at the end; these dictionaries are initialized to `None` but may need to be replaced.

In [ ]:
def makeTree(back, begin, end, A, depth = 0):                    
    """
    Returns a parse given the backtraces in back.
    
    Parameters
    ----------
    
    back: a list of lists where each entry is a dictionary
            each key in the dictionary is the LHS of a nonterminal
    begin : integer start index of the tree to construct
    end : integer end index of the tree to construct
    A: the label of the top node in the current tree
    depth: depth of the recursion, as an integer
        
    Returns
    -------
    a list with an arbitrary structure of embedded lists, indicating
    the best syntactic parse as stored in back

    """
    backptrs = back[begin][end][A]    
    if len(backptrs) == 2:        
        return [A, backptrs[1][0]]        
    elif len(backptrs) == 3:        
        [split, B, C] = backptrs        
        childB = makeTree(back, begin, split, B, depth+1)
        childC = makeTree(back, split+1, end, C, depth+1)
        return [A, [childB, childC]]

In [ ]:
def getHighestProbParse(s, pcfg):
    """
    Returns the highest probability parse and its corresponding probability
    
    Parameters
    ----------
    
    s: the sentence to be parsed
    pcfg: list of tuples that encodes the grammar
        Each list item is a tuple of (input, output, probability)
        input is a string, output is a list of two strings, and
        probability is a float
        
    Returns
    -------
    a dict with two entries. The key 'parse' should contain the highest 
    probability parse, in bracket notation (use makeTree to reconstruct the
    best parse from the variable back). The key 'prob' should contain the 
    probability of the best parse.

    """
    # provided variables
    nonTerms = list(set([x[0] for x in pcfg]))
    numWords = len(s)
    numNonTerms = len(nonTerms)
    # initialize charts, filled with dicts
    d = dict(zip(nonTerms, [0]*len(nonTerms)))
    score = [[d.copy() for x in range(numWords)] for x in range(numWords)] 
    d2 = dict(zip(nonTerms, [None]*len(nonTerms)))
    back = [[d2.copy() for x in range(numWords)] for x in range(numWords)]
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(getHighestProbParse(['his', 'cat', 'runs', 'with', 'a', 'dog'], pcfg))
print(getHighestProbParse(['the','dog','runs','quickly'], pcfg))
print(getHighestProbParse(['the','dog','slowly','barks'], pcfg))
print(getHighestProbParse(['the','dog','quickly','runs'], pcfg))
print(getHighestProbParse(['the','cat','runs','slowly'], pcfg))

In [ ]:
# add your own test cases here!

In [ ]:
from numpy.testing import assert_allclose

# test hard-coded values
testSentences = [['his', 'cat', 'runs', 'with', 'a', 'dog'],
             ['the','dog','runs','quickly'],
             ['the','dog','slowly','barks'],
             ['the','dog','quickly','runs'],
             ['the','cat','runs','slowly']]
testProbabilities = [0.00168, 0.00168, 0.01512, .00672, .01512]

for i in range(len(testSentences)):
    assert_allclose(getHighestProbParse(testSentences[i], pcfg)['prob'], 
        testProbabilities[i])
    
# sample using rewrite string, then check probability of parse. 
# best tree is always >= probability from rewriteString (there may be
# a higher probability way of generating a given string)

for i in range(100):
    if rewriteStringSuccess:        
        testSentence,testProb = rewriteString(['S'],[], pcfg)    
        hpp = getHighestProbParse(testSentence, pcfg)
        assert_allclose(hpp['prob'],testProb)
        #if there is ambiguity in the grammar
        #assert(allclose(hpp['prob'],testProb) or hpp['prob'] < testProb)

print("Success!")

---

Before turning this problem in remember to do the following steps:

1. **Restart the kernel** (Kernel$\rightarrow$Restart)
2. **Run all cells** (Cell$\rightarrow$Run All)
3. **Save** (File$\rightarrow$Save and Checkpoint)

<div class="alert alert-danger">After you have completed these three steps, ensure that the following cell has printed "No errors". If it has <b>not</b> printed "No errors", then your code has a bug in it and has thrown an error! Make sure you fix this error before turning in your problem set.</div>

In [ ]:
print("No errors!")